<a href="https://colab.research.google.com/github/kumaranu7/Recommendation-System-using-NCF/blob/master/Research_paper_implementation_of_NCF_(Neural_Collaborative_Filtering).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [0]:
class NeuMF(torch.nn.Module):
  def __init__(self, config):
    super(NeuMF, self).__init__()

    # matrix_factorization part

    self.embed_user_mf = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_mf)
    self.embed__item_mf = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_mf)
    #Perceptron layer:
    self.embed_user_pl = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_pl)
    self.embed_item_pl = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_pl)

    self.fcl = torch.nn.ModuleList()
    for i, (inputs, outputs) in enumerate(zip(config['layers'][:-1]+ config['layers'][1:])):
      self.fcl.append(torch.nn.Linear(inputs, outputs))

    self.loss = torch.nn.Linear(in_features= config['layers'][-1] + config['latent_mf'], out_features=1)
    self.sigmoid = torch.nn.Sigmoid()

  def forward(self, user, item, title):
    embed_user_pls = self.embed_user_pl(user)
    embed_item_pls = self.embed_item_pl(item)
    embed_user_mfs = self.embed_user_mf(user)
    embed_item_mfs = self.embed_item_mf(user)

    mat_vec = torch.mul(embed_user_pls, embed_item_mfs)
    mat_vec = torch.nn.Dropout(self.config_dropout_rate_mf)(mat_vec) #added extra dropout layer here to increase performance

    pl_vector = torch.cat([embed_user_mfs, embed_item_pls], dim = -1)

    for i, j in enumerate(range(len(self.fcl))):
      pl_vec = self.fcl[i](pl_vec)
      pl_vec = torch.nn.ReLU()(pl_vec)
    
    pl_vec = torch.nn.Dropout(self, config_dropout_rate_lp)(pl_vec) #Added extra dropout layer here to increase performance

    vec = torch.cat([pl_vec, mat_vec], dim = -1)
    loss= self.loss(vec)
    outputs = self.sigmoid(loss)
    return outputs
     